##Start of week 5 - Lasso Regression

In [493]:
import graphlab

In [494]:
sales  =graphlab.SFrame('ML Projects\Course 2\week2\Assignment2\kc_house_data.gl')

In [495]:
from math import log,sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']
##here floors are of type string. So we convert that to float using .astype(float)
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

In [496]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

In [497]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10,verbose = False)

In [498]:
y = model_all.get("coefficients")
print y[y['value']>0]
print model_all["coefficients"]['value'].nnz()

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
+------------------+-------+---------------+--------+
[? rows x 4 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use len(sf) to force materialization.
6


In [499]:
train_valid,test = sales.random_split(.9,seed=1)
train,valid = train_valid.random_split(.5,seed=1)

In [500]:
import numpy as np
l1_array = np.logspace(1,7,num=13)
low_rss=0
l1=0
for l1_penaltys in l1_array:
    model = graphlab.linear_regression.create(train,target = 'price',features=all_features,l1_penalty=l1_penaltys
                                             ,l2_penalty =0.,validation_set=None,verbose=False)
    valid_predict = model.predict(valid)
    error = valid_predict - valid['price']
    rss = (error*error).sum()
    if(l1_penaltys==l1_array[0]):
        low_rss = rss
        l1 = l1_penaltys
        modelx = model
    else:
        if(rss<low_rss):
            low_rss = rss
            l1 = l1_penaltys
            modelx = model
print low_rss
print l1

        

6.25766285142e+14
10.0


In [501]:
modelx["coefficients"]["value"].nnz()

18

In [502]:
model_for_l1 = graphlab.linear_regression.create(train,target = 'price',features=all_features,l1_penalty=10000000.0
                                             ,l2_penalty =0.0,validation_set=None,verbose=False)
test_predict =  model_for_l1.predict(test)
error  =test_predict-test['price']
rss_test= (error*error).sum()
print rss_test

1.56972779669e+14


In [503]:
x = model_for_l1.get("coefficients") 
print len(x[x['value']>0])

17


####Limiting no of non Zero values for coefficients

In [504]:
max_nonzeros = 7

In [505]:
l1_penalty_values = np.logspace(8, 10, num=20)
l1_penalty_values

array([  1.00000000e+08,   1.27427499e+08,   1.62377674e+08,
         2.06913808e+08,   2.63665090e+08,   3.35981829e+08,
         4.28133240e+08,   5.45559478e+08,   6.95192796e+08,
         8.85866790e+08,   1.12883789e+09,   1.43844989e+09,
         1.83298071e+09,   2.33572147e+09,   2.97635144e+09,
         3.79269019e+09,   4.83293024e+09,   6.15848211e+09,
         7.84759970e+09,   1.00000000e+10])

In [506]:
low_rss_l=0
l1_l=0
array_of_non_zeros =[]
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(train,target = 'price',features=all_features,l1_penalty=l1_penalty
                                             ,l2_penalty =0.0,validation_set=None,verbose=False)
#     valid_predict = model.predict(valid)
#     error = valid_predict - valid['price']
#     rss = (error*error).sum()
#     if(l1_penalty==l1_array[0]):
#         low_rss_l = rss
#         l1_l = l1_penalty
#     else:
#         if(rss<low_rss):
#             low_rss_l = rss
#             l1_l = l1_penalty
# print low_rss_l
# print l1_l
    array = model['coefficients']['value']
    array_of_non_zeros.insert(len(array_of_non_zeros),array.nnz())   #nnz() goives us non-xero parameters count
print array_of_non_zeros

        

[18, 18, 18, 18, 17, 17, 17, 17, 17, 16, 15, 15, 13, 12, 10, 6, 5, 3, 1, 1]


In [507]:
array_of_non_zeros.index(10)

14

In [508]:
l1_penalty_min = l1_penalty_values[14]
l1_penalty_min

2976351441.6313128

In [509]:
l1_penalty_max = l1_penalty_values[array_of_non_zeros.index(6)]
l1_penalty_max

3792690190.7322536

###Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set

In [510]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)
l1_penalty_values

array([  2.97635144e+09,   3.01931664e+09,   3.06228184e+09,
         3.10524703e+09,   3.14821223e+09,   3.19117743e+09,
         3.23414263e+09,   3.27710782e+09,   3.32007302e+09,
         3.36303822e+09,   3.40600341e+09,   3.44896861e+09,
         3.49193381e+09,   3.53489901e+09,   3.57786420e+09,
         3.62082940e+09,   3.66379460e+09,   3.70675980e+09,
         3.74972499e+09,   3.79269019e+09])

##### low_rss=0
l1=0
count = 0
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(train,target = 'price',features=all_features,l1_penalty=l1_penalty
                                             ,l2_penalty =0.0,validation_set=None,verbose=False)
    count = model["coefficients"]['value'].nnz()
    valid_predict = model.predict(valid)
    error = valid_predict - valid['price']
    rss = (error*error).sum()
    if(count==max_nonzeros):
        if(low_rss==0):
            low_rss = rss
            l1 = l1_penalty
            model1 = model
        else:
            if(rss<low_rss):
                low_rss = rss
                l1 = l1_penalty
                model1 = model
print low_rss
print l1


In [511]:
x = model1.get("coefficients")
print x[x['value']>0]

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 217416.359436 |  None  |
|     bedrooms     |  None | 1305.30145693 |  None  |
|    bathrooms     |  None |  16849.29857  |  None  |
|   sqft_living    |  None | 33.0614244016 |  None  |
| sqft_living_sqrt |  None | 737.209789863 |  None  |
|      grade       |  None | 3240.64094582 |  None  |
|    sqft_above    |  None | 31.1125668299 |  None  |
+------------------+-------+---------------+--------+
[? rows x 4 columns]
Note: Only the head of the SFrame is printed. This SFrame is lazily evaluated.
You can use len(sf) to force materialization.


###---- End of Week 5 Assignment -2


###Week 5 Main Assingment

In [512]:
import graphlab

In [513]:
sales  =graphlab.SFrame('ML Projects\Course 2\week2\Assignment2\kc_house_data.gl')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

In [514]:
import numpy as np
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1
    #features = ['sqft_living','bedrooms','bathrooms']
    features = ['constant']+features  #added constant to features 
    feature_sframe = data_sframe[features]  #Contains only the features column data
    features_matrix = feature_sframe.to_numpy()  #converting it into array using numpy
    output_sarray = data_sframe['price'] #here price is the output
    output_array = output_sarray.to_numpy() #conveting to numpy array
    return(features_matrix,output_array)

In [515]:
def predict_output(featured_matrix,weights):
    dot_product_matrix = np.dot(featured_matrix,weights)
    return dot_product_matrix

In [516]:
##Example
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


In [517]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[  5.  13.  17.]


In [518]:
print X / norms

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


In [519]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix,axis=0)
    return (feature_matrix/norms,norms)

In [520]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
print norms

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


###Implementing coordinate decent by normalizing the values

In [521]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

In [522]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

In [523]:
weights = np.array([1., 4., 1.])

In [524]:
prediction = predict_output(simple_feature_matrix,weights)
print len(simple_feature_matrix)
print len(weights)

21613
3


In [529]:
def ro(features_matrix,output,prediction,weights):
    i=0
    ro  =[]
    while (i<len(weights)):
        ro.insert(i,(features_matrix[:,i]*(output-prediction+(weights[i]*features_matrix[:,i]))).sum())
        i = i+1
    return ro

In [530]:
ro(simple_feature_matrix,output,prediction,weights)   #--- This is correct 

[79400300.034929156, 87939470.772991076, 80966698.675965652]

### Single Coordinate decent steps

In [722]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    simple_feature_matrix_c, norms_c = normalize_features(feature_matrix)
    predictions_c = predict_output(simple_feature_matrix_c,weights)
    ro_c =  (simple_feature_matrix_c[:,i]*(output-predictions_c+weights[i]*simple_feature_matrix_c[:,i])).sum()
    print "ro_c:",ro_c
    l1_half =l1_penalty/2.
    print l1_half
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_c
    elif (ro_c < (-1*l1_half)):
        new_weight_i = (ro_c + l1_half)
    elif (ro_c > l1_half):
        ##print "in here"
        ##print (ro_c - l1_half)
        new_weight_i = (ro_c - l1_half)
    else:
        new_weight_i = 0.
    print "new weight :",new_weight_i
    print "**********"
    return new_weight_i

In [621]:
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


In [622]:
#--- above lasso_coordinate_descent_step function is working fine 

###Cyclical coordinate descent

In [698]:
def modulus(mod):
    if(mod<0):
        return mod*-1
    else:
        return mod

In [723]:
def lasso_cyclical_coordinate_descent(feature_matrix_f, output_f, initial_weights_f, l1_penalty_f, tolerance_f):
    max_change = 1e6
    weights =initial_weights_f
    ##print weights[0]
    while(max_change>tolerance_f):
        max_change=0
        change_per_loop=0
        for i in range(len(weights)):
            old_weight_i = weights[i]
            weights[i]=lasso_coordinate_descent_step(i, feature_matrix_f, output_f, weights, l1_penalty_f)
            print weights[i]
            if(i==0):
                change_per_loop = modulus(weights[i]-old_weight_i)
            else :
                if(change_per_loop<(modulus(weights[i]-old_weight_i))):
                    change_per_loop = modulus(weights[i]-old_weight_i)
        print "change per loop :",change_per_loop
        max_change = change_per_loop
    return weights

In [700]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0
initial_weights

array([ 0.,  0.,  0.])

In [701]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features
normalized_simple_feature_matrix

array([[ 0.00680209,  0.00353021,  0.00583571],
       [ 0.00680209,  0.00768869,  0.00583571],
       [ 0.00680209,  0.00230361,  0.00389048],
       ..., 
       [ 0.00680209,  0.00305154,  0.00389048],
       [ 0.00680209,  0.00478673,  0.00583571],
       [ 0.00680209,  0.00305154,  0.00389048]])

In [702]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

0.0
change per loop : 79400304.6581
change per loop : 9138168.3665
change per loop : 8194809.40937
change per loop : 6598905.03827
change per loop : 5522173.1943
change per loop : 4621129.81031
change per loop : 3867108.10625
change per loop : 3236118.81061
change per loop : 2708086.93955
change per loop : 2266213.1094
change per loop : 1896439.06265
change per loop : 1587000.40319
change per loop : 1328052.31095
change per loop : 1111356.328
change per loop : 930018.251241
change per loop : 778268.79269
change per loop : 651280.029038
change per loop : 545011.800818
change per loop : 456083.174344
change per loop : 381664.876995
change per loop : 319389.283634
change per loop : 267275.090398
change per loop : 223664.279321
change per loop : 187169.368346
change per loop : 156629.268444
change per loop : 131072.343461
change per loop : 109685.497423
change per loop : 91788.3058116
change per loop : 76811.3677895
change per loop : 64278.190664
change per loop : 53790.0302253
change per 

In [703]:
print weights

[ 21624998.36636282  63157246.7854543          0.        ]


In [704]:
output_predicted = predict_output(normalized_simple_feature_matrix,weights)
error = output_predicted - output
rss = (error*error).sum()
print rss


1.63049248148e+15


### Evaluating LASSO fit with more features

In [705]:
train_data,test_data = sales.random_split(.8,seed=0)

In [706]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

In [707]:
(features_matrix_train,output_train) = get_numpy_data(train_data,all_features,'price') 
(normalize_features_matrix_train,norms_train) = normalize_features(features_matrix_train)

In [708]:
initial_weights_train = np.zeros(14)
l1_penalty_train = 1e7
tolerance_train = 1.0
weights_1e7 = lasso_cyclical_coordinate_descent(normalize_features_matrix_train,output_train,
                                                initial_weights_train,1e7,1)
norm_1e7 = weights_1e7/norms_train
print weights_1e7
print norm_1e7

0.0
change per loop : 71114625.7528
change per loop : 5024356.36869
change per loop : 5012324.91395
change per loop : 5000676.42086
change per loop : 4991243.17314
change per loop : 4984361.34335
change per loop : 4979504.92921
change per loop : 4973806.56525
change per loop : 2879222.99938
change per loop : 2706150.76622
change per loop : 2532295.64864
change per loop : 2368316.57782
change per loop : 2213124.35067
change per loop : 1552443.60013
change per loop : 507838.033159
change per loop : 438814.370591
change per loop : 371479.622608
change per loop : 313415.105581
change per loop : 264244.832139
change per loop : 222753.8608
change per loop : 187770.667306
change per loop : 158280.093357
change per loop : 133420.901606
change per loop : 112465.988296
change per loop : 94802.2134519
change per loop : 79912.6874505
change per loop : 67361.6926196
change per loop : 56781.942492
change per loop : 47863.835752
change per loop : 40346.3966195
change per loop : 34009.6378518
change p

In [724]:
#Now with l1_penalty_train = 1e7
initial_weights_train = np.zeros(14)
l1_penalty_train = 1e8
weights_1e8 = lasso_cyclical_coordinate_descent(normalize_features_matrix_train,output_train,
                                                initial_weights_train,1e8,1)
norm_1e8 = weights_1e8/norms_train
print weights_1e8
print norm_1e8

ro_c: 71114625.7528
50000000.0
new weight : 71114625.7528
**********
71114625.7528
ro_c: 3961380.1017
50000000.0
new weight : 0.0
**********
0.0
ro_c: 8743972.43192
50000000.0
new weight : 0.0
**********
0.0
ro_c: 13878982.8537
50000000.0
new weight : 0.0
**********
0.0
ro_c: 4041432.70443
50000000.0
new weight : 0.0
**********
0.0
ro_c: 4042646.85757
50000000.0
new weight : 0.0
**********
0.0
ro_c: 13219102.7965
50000000.0
new weight : 0.0
**********
0.0
ro_c: 18217825.1197
50000000.0
new weight : 0.0
**********
0.0
ro_c: 329621.962263
50000000.0
new weight : 0.0
**********
0.0
ro_c: 4913611.00922
50000000.0
new weight : 0.0
**********
0.0
ro_c: 12386943.3964
50000000.0
new weight : 0.0
**********
0.0
ro_c: 13490052.6687
50000000.0
new weight : 0.0
**********
0.0
ro_c: 38085.1956153
50000000.0
new weight : 0.0
**********
0.0
ro_c: 5893049.59967
50000000.0
new weight : 0.0
**********
0.0
change per loop : 71114625.7528
ro_c: 71114625.7528
50000000.0
new weight : 71114625.7528
*********

In [710]:
#now with l1_penalty_train=1e4 and tolerance_train=5e5
initial_weights_train = np.zeros(14)
l1_penalty_train=1e4
tolerance_train=5e5
weights_1e4 = lasso_cyclical_coordinate_descent(normalize_features_matrix_train,output_train,
                                                initial_weights_train,1e4,5e5)
norm_1e4 = weights_1e4/norms_train
print weights_1e4
print norm_1e4

0.0
change per loop : 71114625.7528
change per loop : 8464015.94738
change per loop : 6818451.35869
change per loop : 5570165.21901
change per loop : 4559838.46759
change per loop : 3732056.69715
change per loop : 3053211.61353
change per loop : 2497517.43558
change per loop : 2043868.97325
change per loop : 1674717.70906
change per loop : 1375420.75692
change per loop : 1296621.8143
change per loop : 1257340.9361
change per loop : 1228271.24819
change per loop : 1206854.36471
change per loop : 1191059.40598
change per loop : 1179030.77017
change per loop : 1169777.43345
change per loop : 1162340.67589
change per loop : 1155980.52901
change per loop : 1150139.67966
change per loop : 1144406.32752
change per loop : 1138483.91805
change per loop : 1132166.57169
change per loop : 1125319.16778
change per loop : 1117861.24888
change per loop : 1109754.06282
change per loop : 1100990.17907
change per loop : 1091585.21244
change per loop : 1081571.26761
change per loop : 1070991.78405
change

In [711]:
# original_features = train[all_features].to_numpy()
# features,norms = normalize_features(original_features)
test_feature_matrix, test_output = get_numpy_data(test_data, all_features, 'price')
# weights1_1e7 = lasso_cyclical_coordinate_descent(normalize_features_matrix_train,output_train,
#                                                 initial_weights_train,1e7,1.0)

In [712]:
#w1
print norm_1e7
predict_w1 = predict_output(test_feature_matrix,norm_1e7)
error = predict_w1-test_output
rss_w1 = (error*error).sum()
print rss_w1

[  1.85285533e+05   0.00000000e+00   0.00000000e+00   1.61317456e+02
   0.00000000e+00   0.00000000e+00   2.87664700e+05   6.91937057e+04
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00]
2.75962079909e+14


In [713]:
#w2
print norm_1e8
predict_w2 = predict_output(test_feature_matrix,norm_1e8)
error = predict_w2-test_output
rss_w2 = (error*error).sum()
print rss_w2

[ 539366.62822135       0.               0.               0.               0.
       0.               0.               0.               0.               0.
       0.               0.               0.               0.        ]
5.37166150034e+14


In [714]:
#w3
print norm_1e4
predict_w3 = predict_output(test_feature_matrix,norm_1e4)
error = predict_w3-test_output
rss_w3 = (error*error).sum()
print rss_w3

[  5.89912924e+05  -4.97435039e+04   5.17044250e+04   3.07261390e+02
  -3.67765574e-01  -4.32048893e+04   5.63119400e+05   6.66940353e+04
   8.99767715e+03   1.80569342e+04  -5.60846894e+01  -7.88384489e+01
  -3.21603081e+02   5.18531810e+01]
2.2778100476e+14
